# 一、数据预处理

将VOC标注的标注文件.xml转为COCO格式的.json文件

在训练目标检测模型的时候一般使用labelimg标注的图像生产.xml格式的标注文件。有时候需要用到coco格式的json标注文件，在github找到了一个xml转json的脚本。(https://github.com/CivilNet/Gemfield/blob/master/src/python/pascal_voc_xml2json/pascal_voc_xml2json.py)

执行该脚本会读取Annotations下的.xml文件并解析其中的类别及boundbox的坐标，最后生成instances.json的文件。

原git的脚本不支持划分训练集测试集，也不能提前设定标签的索引。所以进行了改动，如下：

（将所有图片和xml文件都放到同一个文件目录下，此处为/home/ouc/xyz/kdxf/train/TIFFImages/）

In [ ]:
#coding:utf-8
 
import os
import glob
import json
import shutil
import numpy as np
import xml.etree.ElementTree as ET
 
path2 = "."
 
START_BOUNDING_BOX_ID = 1
 
def get(root, name):
    return root.findall(name)
 
def get_and_check(root, name, length):
    vars = root.findall(name)
    if len(vars) == 0:
        raise NotImplementedError('Can not find %s in %s.'%(name, root.tag))
    if length > 0 and len(vars) != length:
        raise NotImplementedError('The size of %s is supposed to be %d, but is %d.'%(name, length, len(vars)))
    if length == 1:
        vars = vars[0]
    return vars
 
def convert(xml_list, json_file):
    json_dict = {"images": [], "type": "instances", "annotations": [], "categories": []}
    categories = pre_define_categories.copy()
    bnd_id = START_BOUNDING_BOX_ID
    all_categories = {}
    for index, line in enumerate(xml_list):
        xml_f = line
        tree = ET.parse(xml_f)
        root = tree.getroot()
        
        filename = os.path.basename(xml_f)[:-4] + ".tif"
        image_id = index
        size = get_and_check(root, 'size', 1)
        width = int(get_and_check(size, 'width', 1).text)
        height = int(get_and_check(size, 'height', 1).text)
        image = {'file_name': filename, 'height': height, 'width': width, 'id':image_id}
        json_dict['images'].append(image)
        ## Cruuently we do not support segmentation
        #  segmented = get_and_check(root, 'segmented', 1).text
        #  assert segmented == '0'
        for obj in get(root, 'object'):
            category = get_and_check(obj, 'name', 1).text
            if category in all_categories:
                all_categories[category] += 1
            else:
                all_categories[category] = 1
            if category not in categories:
                if only_care_pre_define_categories:
                    continue
                new_id = len(categories) + 1
                print("[warning] category '{}' not in 'pre_define_categories'({}), create new id: {} automatically".format(category, pre_define_categories, new_id))
                categories[category] = new_id
            category_id = categories[category]
            bndbox = get_and_check(obj, 'bndbox', 1)
            xmin = int(float(get_and_check(bndbox, 'xmin', 1).text))
            ymin = int(float(get_and_check(bndbox, 'ymin', 1).text))
            xmax = int(float(get_and_check(bndbox, 'xmax', 1).text))
            ymax = int(float(get_and_check(bndbox, 'ymax', 1).text))
            assert(xmax > xmin), "xmax <= xmin, {}".format(line)
            assert(ymax > ymin), "ymax <= ymin, {}".format(line)
            o_width = abs(xmax - xmin)
            o_height = abs(ymax - ymin)
            ann = {'area': o_width*o_height, 'iscrowd': 0, 'image_id':
                   image_id, 'bbox':[xmin, ymin, o_width, o_height],
                   'category_id': category_id, 'id': bnd_id, 'ignore': 0,
                   'segmentation': []}
            json_dict['annotations'].append(ann)
            bnd_id = bnd_id + 1
 
    for cate, cid in categories.items():
        cat = {'supercategory': 'none', 'id': cid, 'name': cate}
        json_dict['categories'].append(cat)
    json_fp = open(json_file, 'w')
    json_str = json.dumps(json_dict)
    json_fp.write(json_str)
    json_fp.close()
    print("------------create {} done--------------".format(json_file))
    print("find {} categories: {} -->>> your pre_define_categories {}: {}".format(len(all_categories), all_categories.keys(), len(pre_define_categories), pre_define_categories.keys()))
    print("category: id --> {}".format(categories))
    print(categories.keys())
    print(categories.values())
 
if __name__ == '__main__':
    classes = ['Port', 'Airport']
    pre_define_categories = {}
    for i, cls in enumerate(classes):
        pre_define_categories[cls] = i + 1
    only_care_pre_define_categories = True
 
    train_ratio = 0.9
    save_json_train = 'instances_train2014.json'
    save_json_val = 'instances_val2014.json'
    xml_dir = "/home/ouc/xyz/kdxf/train/TIFFImages/"
 
    xml_list = glob.glob(xml_dir + "/*.xml")
    xml_list = np.sort(xml_list)
    np.random.seed(100)
    np.random.shuffle(xml_list)
 
    train_num = int(len(xml_list)*train_ratio)
    xml_list_train = xml_list[:train_num]
    xml_list_val = xml_list[train_num:]
 
    convert(xml_list_train, save_json_train)
    convert(xml_list_val, save_json_val)
 
    if os.path.exists(path2 + "/annotations"):
        shutil.rmtree(path2 + "/annotations")
    os.makedirs(path2 + "/annotations")
    if os.path.exists(path2 + "/images/train2014"):
        shutil.rmtree(path2 + "/images/train2014")
    os.makedirs(path2 + "/images/train2014")
    if os.path.exists(path2 + "/images/val2014"):
        shutil.rmtree(path2 +"/images/val2014")
    os.makedirs(path2 + "/images/val2014")
 
    f1 = open("train.txt", "w")
    for xml in xml_list_train:
        img = xml[:-4] + ".tif"
        f1.write(os.path.basename(xml)[:-4] + "\n")
        shutil.copyfile(img, path2 + "/images/train2014/" + os.path.basename(img))
 
    f2 = open("test.txt", "w")
    for xml in xml_list_val:
        img = xml[:-4] + ".tif"
        f2.write(os.path.basename(xml)[:-4] + "\n") 
        shutil.copyfile(img, path2 + "/images/val2014/" + os.path.basename(img))
    f1.close()
    f2.close()
    print("-------------------------------")
    print("train number:", len(xml_list_train))
    print("val number:", len(xml_list_val))

# 二、安装 YOLOX

1. 安装 YOLOX

In [ ]:
# 安装YOLOX
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
!pip3 install -r YOLOX/requirements.txt
!cd YOLOX && pip install -e .

2. 安装 apex & pycocotools

In [ ]:
# 安装apex
# !git clone https://github.com/NVIDIA/apex
# !cd apex && pip3 install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_exp" --global-option="--cuda_ext" ./
# 用上面方法安装apex失败，采取下方方式安装
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./
# 安装pycocotools
!pip3 install cython
!pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# 三、将数据集放到YOLOX/data/datasets下

In [ ]:
# 在 YOLOX 文件夹下创建相关文件夹
!mkdir YOLOX/datasets/COCO
!mkdir YOLOX/datasets/COCO/annotations

In [ ]:
# 将 instances_train2014.json & instances_val2014.json 文件复制到 YOLOX 文件夹下 并重新取名
!cp 'instances_train2014.json' YOLOX/datasets/COCO/annotations/instances_train2017.json
!cp 'instances_val2014.json' YOLOX/datasets/COCO/annotations/instances_val2017.json
!ls YOLOX/datasets/COCO/annotations

In [ ]:
# 将数据集文件夹 映射 到 YOLOX 文件夹下，并重新取名（映射不占多余空间）
!ln -s '/home/ouc/xyz/kdxf/train/TIFFImages/' YOLOX/datasets/COCO/train2017
!ln -s '/home/ouc/xyz/kdxf/train/TIFFImages/' YOLOX/datasets/COCO/val2017

# 四、准备我们自己的 YOLOX exp & cococlasses
1. 设置我们自己的 exp, 这里用 yolox_x

In [ ]:
!echo "        self.num_classes = 2" >> YOLOX/exps/default/yolox_x.py # 修改类别数
!echo "        self.max_epoch = 10" >> YOLOX/exps/default/yolox_x.py # 修改epochs
!echo "        self.eval_interval = 1" >> YOLOX/exps/default/yolox_x.py
!echo "        self.warmup_epochs = 2" >> YOLOX/exps/default/yolox_x.py

!cat YOLOX/exps/default/yolox_x.py

2. 设置 coco_classes

In [ ]:
!echo "COCO_CLASSES = ('Port','Airport')" > "YOLOX/yolox/data/datasets/coco_classes.py"
!cat YOLOX/yolox/data/datasets/coco_classes.py

# 五、下载预训练模型

In [ ]:
!wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_x.pth
!mv yolox_x.pth yolox_x.pth.tar

# 六、用预训练模型进行训练

In [ ]:
!python YOLOX/tools/train.py -f YOLOX/exps/default/yolox_x.py -d 1 -b 4 --fp16 -o -c yolox_x.pth.tar

# 七、评估

In [ ]:
!python YOLOX/tools/eval.py -f YOLOX/exps/default/yolox_x.py -c YOLOX_outputs/yolox_x/latest_ckpt.pth -b 4 -d 1 --conf 0.001 --fp16 --fuse

# 八、可视化 demo

In [ ]:
!mv YOLOX_outputs/yolox_x/latest_ckpt.pth YOLOX_outputs/yolox_x/latest_ckpt.pth.tar

In [ ]:
!python YOLOX/tools/demo.py image -n yolox-x -c YOLOX_outputs/yolox_x/latest_ckpt.pth.tar --path '/home/ouc/xyz/kdxf/train/TIFFImages/0.tif' --conf 0.25 --nms 0.45 --tsize 640 --save_result --device gpu

# 九、Output & Submit
1. 定义 predict 函数来返回 output

In [ ]:
def predict(pth,jpg):
    %cd YOLOX
    from yolox.exp import get_exp
    from loguru import logger
    from yolox.utils import fuse_model, get_model_info, postprocess, vis
    from yolox.data.data_augment import preproc
    import torch,cv2
    exp=get_exp('/exps/default/yolox_x.py','yolox_x')
    model = exp.get_model()
    #logger.info("Model Summary: {}".format(get_model_info(model, exp.test_size)))
    model.cuda()
    model.eval()
    ckpt_file=pth
    ckpt = torch.load(ckpt_file, map_location="cpu")
    model.load_state_dict(ckpt["model"])
    model = fuse_model(model)
    img = cv2.imread(jpg)
    img, ratio = preproc(img, exp.test_size, (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # test_size = (640, 640)
    img = torch.from_numpy(img).unsqueeze(0)
    img = img.cuda()
    outputs = model(img)
    #outputs = postprocess(outputs, 5, exp.test_conf, exp.nmsthre) #test_conf = 0.01 nmsthre = 0.65
    outputs = postprocess(outputs, 5, 0.25 , 0.45)
    output = outputs[0]
    if output==None:
        %cd ..
        return None,None,None
    output = output.cpu()
    bboxes = output[:, 0:4]
    bboxes=bboxes/ratio
    cls = output[:, 6]
    scores = output[:, 4] * output[:, 5]
    %cd ..
    return bboxes,cls,scores

In [ ]:
pth='YOLOX_outputs/yolox_x/latest_ckpt.pth.tar'

2. 用 output 来返回 submissions